### Neural Machine Translation using word level language model and embeddings in Keras

In [1]:
%pwdㅎ

'D:\\Visual_Storytelling_Challenge2018\\code\\IBM'

In [2]:
import pandas as pd
import numpy as np
import string
import pickle
from pandas import Series
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.cross_validation import train_test_split
# Building a english to french translator

C:\Users\USER\Anaconda3\envs\gpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
# load
with open('df_story_desc_final.pickle', 'rb') as f:
    df_story_and_desc_pickle = pickle.load(f)

df_story_and_desc = df_story_and_desc_pickle
print ("before: ", df_story_and_desc.shape)
df_story_and_desc = df_story_and_desc[:30000]
df_story_and_desc_id = df_story_and_desc['Story_Photo_id']
df_story_and_desc_text = df_story_and_desc[['Desc', 'Story']]
print (type(df_story_and_desc_text))
print (df_story_and_desc_text.shape)
df_story_and_desc_text.head(10)

before:  (145116, 4)
<class 'pandas.core.frame.DataFrame'>
(30000, 2)


,Desc,Story
0,big old tree being photographed on a sunny day,"and its magnificent trunk , larger than life i..."
1,a old curvy tree in the sun light .,"and its magnificent trunk , larger than life i..."
2,a person is taking a picture of a large tree a...,"and its magnificent trunk , larger than life i..."
3,large tree with many outstretching branches an...,we found this tree when we were walking in a n...
4,a green sign is describing a historic tree and...,it turns out it is a popular attraction here .
5,a large tree with roots that look like crocodi...,"the tree is very unusual , with its roots expo..."
6,big old tree being photographed on a sunny day,"the trunk was really wide , as much as 12 feet !"
7,huge brown tree roots rose above the ground .,you can see how big these roots are - pretty a...
8,a large tree with many branches coming out,we found this tree when we were walking in a n...
9,a plaque describes an historical tree and advi...,it turns out it is a popular attraction here .


In [5]:
df_story_and_desc_text.iloc[4000].Desc

'the people shoot fireworks into the air at night .'

In [6]:
df_story_and_desc_text.iloc[4000].Story

'they had some low flying fireworks at the display that evening .'

# Data cleaning
<br />

In [7]:
def re_sub(item):
    re_sentence = []
    for sentence in item:
        sentence = re.sub(" '", "'", sentence) # 'll, 'm, 's
        sentence = re.sub(" ,", ",", sentence)
        sentence = re.sub(" [.]", ". ", sentence)
        sentence = re.sub(" n't", "n't",sentence)
        sentence = re.sub(" !", "! ", sentence)
        sentence = re.sub("  ", " ", sentence)
        sentence = sentence[:-1]
        re_sentence.append(sentence)
    return re_sentence

In [8]:
clean_df_all = df_story_and_desc_text.apply(lambda x: re_sub(x))
clean_df_all.columns = ['description_text', 'story_text']
clean_df_all

,description_text,story_text
0,big old tree being photographed on a sunny da,"and its magnificent trunk, larger than life it..."
1,a old curvy tree in the sun light.,"and its magnificent trunk, larger than life it..."
2,a person is taking a picture of a large tree a...,"and its magnificent trunk, larger than life it..."
3,large tree with many outstretching branches an...,we found this tree when we were walking in a n...
4,a green sign is describing a historic tree and...,it turns out it is a popular attraction here.
5,a large tree with roots that look like crocodi...,"the tree is very unusual, with its roots exposed."
6,big old tree being photographed on a sunny da,"the trunk was really wide, as much as 12 feet!"
7,huge brown tree roots rose above the ground.,you can see how big these roots are - pretty a...
8,a large tree with many branches coming ou,we found this tree when we were walking in a n...
9,a plaque describes an historical tree and advi...,it turns out it is a popular attraction here.


In [9]:
clean_df_all.story_text[2000]

'getting the kids together to play was long overdue.'

In [10]:
clean_df_all.story_text.head()

0    and its magnificent trunk, larger than life it...
1    and its magnificent trunk, larger than life it...
2    and its magnificent trunk, larger than life it...
3    we found this tree when we were walking in a n...
4        it turns out it is a popular attraction here.
Name: story_text, dtype: object

# Generate synthetic data
<br / >

In [11]:
print (type(clean_df_all.description_text))
clean_df_all.description_text[:5]

<class 'pandas.core.series.Series'>


0        big old tree being photographed on a sunny da
1                   a old curvy tree in the sun light.
2    a person is taking a picture of a large tree a...
3    large tree with many outstretching branches an...
4    a green sign is describing a historic tree and...
Name: description_text, dtype: object

In [12]:
clean_df_all.story_text = clean_df_all.story_text.apply(lambda x : 'START_ '+ x + ' _END')
print (type(clean_df_all.story_text))
clean_df_all.story_text[:5]

<class 'pandas.core.series.Series'>


0    START_ and its magnificent trunk, larger than ...
1    START_ and its magnificent trunk, larger than ...
2    START_ and its magnificent trunk, larger than ...
3    START_ we found this tree when we were walking...
4    START_ it turns out it is a popular attraction...
Name: story_text, dtype: object

In [13]:
all_desc_words = set()
for desc_paragraph in clean_df_all.description_text:
    for word in desc_paragraph.split():
        if word not in all_desc_words:
            all_desc_words.add(word)
            
all_story_words = set()
for story_parapgraph in clean_df_all.story_text:
    for word in story_parapgraph.split():
        if word not in all_story_words:
            all_story_words.add(word)
    
len(all_desc_words), len(all_story_words)

(7863, 9940)

In [14]:
desc_lenght_list=[]
for desc_paragraph in clean_df_all.description_text:
    desc_lenght_list.append(len(desc_paragraph.split(' ')))

max_desc_length = np.max(desc_lenght_list)
max_desc_length

54

In [15]:
story_lenght_list=[]
for story_parapgraph in clean_df_all.story_text:
    story_lenght_list.append(len(story_parapgraph.split(' ')))

max_story_length = np.max(story_lenght_list)
max_story_length

82

In [16]:
input_words = sorted(list(all_desc_words))
target_words = sorted(list(all_story_words))
num_encoder_tokens = len(all_desc_words)
num_decoder_tokens = len(all_story_words)
print ("num input_words: {0}, num target_words: {1}".format(num_encoder_tokens, num_decoder_tokens))
# del all_eng_words, all_french_words

num input_words: 7863, num target_words: 9940


In [17]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [18]:
input_token_index

{'chef': 1359,
 'lapse.': 3862,
 'glassed': 3044,
 'stormy': 6651,
 'frie': 2887,
 'bearing': 635,
 'button': 1091,
 'attire,': 434,
 'schoolhouse': 5872,
 'uneaten': 7369,
 'treehous': 7238,
 'intersectio': 3650,
 'load.': 4046,
 'foregroun': 2827,
 'retro': 5607,
 'delapidated': 1975,
 'bounce.': 889,
 'steel': 6587,
 'watch.': 7582,
 'trip': 7262,
 'crop': 1819,
 'reluctant': 5558,
 'fur.': 2933,
 'smoky': 6302,
 'gas': 2968,
 'circles': 1427,
 'parka': 4866,
 'shepard.': 6021,
 'cellular': 1284,
 'wispy': 7746,
 'eagerly': 2286,
 'basketball': 596,
 'illustration': 3564,
 'for': 2822,
 'shiny.': 6035,
 'case.': 1235,
 'mug': 4458,
 'skylin': 6227,
 'modern': 4369,
 'sharing': 6000,
 'drums.': 2251,
 'practice': 5281,
 'bottom.': 882,
 'deck.|a': 1955,
 'handle': 3264,
 'water,': 7594,
 'base.': 586,
 'antenna': 293,
 'streaking': 6678,
 'attention.': 431,
 'swirly': 6857,
 'property.': 5359,
 'advertises': 149,
 'newspaper': 4560,
 'ground.': 3194,
 'luxury': 4138,
 '200': 17,
 'br

In [19]:
len(clean_df_all.story_text)*16*num_decoder_tokens

4771200000

In [20]:
encoder_input_data = np.zeros(
    (len(clean_df_all.description_text), max_desc_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(clean_df_all.story_text), max_story_length),
     dtype='float32')
decoder_target_data = np.zeros(
    (len(clean_df_all.story_text), max_story_length, num_decoder_tokens),
    dtype='float32')
# decoder_target_data = np.zeros(
#     (len(clean_df_all.story_text), 500),
#     dtype='float32')

In [21]:
print (encoder_input_data.shape)
encoder_input_data

(30000, 54)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [22]:
print (decoder_input_data.shape)
decoder_input_data

(30000, 82)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [23]:
print (decoder_target_data.shape)
decoder_target_data

(30000, 82, 9940)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [24]:
decoder_input_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
target_token_index

{'chef': 1672,
 'favorite.': 3356,
 'the,': 8814,
 'complete,': 1977,
 'possibility': 6685,
 'out,': 6151,
 'admired': 176,
 'stormy': 8388,
 'prom.': 6829,
 'ther': 8834,
 'start!': 8282,
 'schoolhouse': 7510,
 'gasp.': 3785,
 'exit.': 3176,
 'dousing': 2729,
 'standby': 8264,
 'highway.': 4338,
 'ponds.': 6647,
 "didn't": 2564,
 'bangs,': 740,
 'swim,': 8633,
 'cocktail': 1876,
 'knocking': 4956,
 'gem': 3813,
 'views,': 9435,
 'anticipation': 399,
 'architecture,': 475,
 'reluctant': 7143,
 'eat!': 2884,
 'upsetting': 9336,
 'victory': 9422,
 'uh-oh,': 9259,
 'gas': 3781,
 'cultures': 2300,
 'wow': 9850,
 'farther': 3334,
 'celebtated': 1578,
 'gorgeous,': 3962,
 'perfect.': 6410,
 'basketball': 780,
 'await': 628,
 'reenactments': 7092,
 'intend': 4700,
 'for': 3615,
 'degree.': 2468,
 'mug': 5741,
 'contort': 2079,
 'attendees': 590,
 'characters,': 1634,
 'ea': 2854,
 'modern': 5634,
 'dangling': 2366,
 'roofs': 7342,
 'help.': 4288,
 'practice': 6716,
 '2nd': 43,
 'bottom.': 113

In [26]:
# clean_df_all.description_text, clean_df_all.story_text
for i, (input_text, target_text) in enumerate(zip(clean_df_all.description_text, clean_df_all.story_text)):
    #print ("input_text: {0}, target_text: {1}".format(input_text, target_text))
    # encoder
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    # decoder
    try:
        for t, word in enumerate(target_text.split()):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t] = target_token_index[word]  # word_index를 decoder_input_data에 넣음.
            #print ("check")
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[word]] = 1.
    except Exception as e:
        print (e)
        print ("t number: ", t)

In [27]:
print (decoder_input_data.shape)
decoder_input_data

(30000, 82)


array([[  72.,  361., 4789., ...,    0.,    0.,    0.],
       [  72.,  361., 4789., ...,    0.,    0.,    0.],
       [  72.,  361., 4789., ...,    0.,    0.,    0.],
       ...,
       [  72., 8813., 4919., ...,    0.,    0.,    0.],
       [  72., 8843., 5295., ...,    0.,    0.,    0.],
       [  72., 9603., 4139., ...,    0.,    0.,    0.]], dtype=float32)

In [28]:
print(decoder_target_data.shape)
decoder_target_data

(30000, 82, 9940)


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# Build keras encoder-decoder model

In [29]:
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints


class AttentionL(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(AttentionL, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

    def get_config(self):
        config={'step_dim':self.step_dim}
        base_config = super(AttentionL, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

C:\Users\USER\Anaconda3\envs\gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
from keras.layers import Input, LSTM, Embedding, Dense, Concatenate, Bidirectional, RepeatVector, concatenate, TimeDistributed, dot
from keras.models import Model
from keras.utils import plot_model
from keras import Input

In [31]:
EMBEDDING_DIM = 150
vocab_size_input = num_encoder_tokens
vocab_size_output = num_decoder_tokens
MAX_LENGTH_INPUT = max_desc_length
MAX_LENGTH_OUTPUT = max_story_length
units = 100

# Model test

In [32]:
# 11-02  decoder 1 layer
encoder_inputs = Input(shape=(None,))

display('encoder_inputs: ', encoder_inputs.shape)

en_x=  Embedding(num_encoder_tokens, EMBEDDING_DIM)(encoder_inputs)

encoder = Bidirectional(LSTM(units, return_state=True,
                             dropout = 0.5, recurrent_dropout = 0.5))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(en_x)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

### decoder

decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, EMBEDDING_DIM)

final_dex= dex(decoder_inputs)

decoder_lstm = LSTM(units * 2, return_sequences=True, return_state=True,
                    dropout = 0.5, recurrent_dropout = 0.5)

print (decoder_lstm(final_dex, initial_state = encoder_states))
decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state = encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

'encoder_inputs: '

TensorShape([Dimension(None), Dimension(None)])

[<tf.Tensor 'lstm_2/transpose_1:0' shape=(?, ?, 200) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_2:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'lstm_2/while/Exit_3:0' shape=(?, 200) dtype=float32>]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 150)    1179450     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
bidirec

In [33]:
import keras.backend.tensorflow_backend as K
with K.tf.device('/gpu:0'):
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size = 128,
              epochs = 50)

Epoch 1/50
30000/30000 [==============================] - 423s 14ms/step - loss: 0.9122 - acc: 0.0163
Epoch 2/50
30000/30000 [==============================] - 455s 15ms/step - loss: 0.8310 - acc: 0.0216
Epoch 3/50
30000/30000 [==============================] - 409s 14ms/step - loss: 0.7771 - acc: 0.0259
Epoch 4/50
30000/30000 [==============================] - 403s 13ms/step - loss: 0.7445 - acc: 0.0284
Epoch 5/50
30000/30000 [==============================] - 411s 14ms/step - loss: 0.7179 - acc: 0.0306
Epoch 6/50
30000/30000 [==============================] - 406s 14ms/step - loss: 0.6934 - acc: 0.0326
Epoch 7/50
30000/30000 [==============================] - 426s 14ms/step - loss: 0.6716 - acc: 0.0338
Epoch 8/50
30000/30000 [==============================] - 407s 14ms/step - loss: 0.6520 - acc: 0.0348
Epoch 9/50
30000/30000 [==============================] - 415s 14ms/step - loss: 0.6342 - acc: 0.0358
Epoch 10/50
30000/30000 [==============================] - 424s 14ms/step - loss: 

In [77]:
# from keras.models import load_model
# model.save("1124_bilstm_emb150_model22.h5")

C:\Users\USER\Anaconda3\envs\gpu\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1/concat:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'concatenate_2/concat:0' shape=(?, 200) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


#### Create sampling model

In [34]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 150)    1179450     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) [(None, 200), (None, 200800      embedding_1[0][0]                
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 200)          0           bidirectional_1[0][1]            
                                                                 bidirectional_1[0][3]            
__________

In [35]:
decoder_state_input_h = Input(shape=(units * 2,))  # encoder를 bilstm으로 학습했기 때문에 shape이 50이 아니고 100이다.
decoder_state_input_c = Input(shape=(units * 2,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)
print (final_dex2)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


Tensor("embedding_2_1/embedding_lookup:0", shape=(?, ?, 150), dtype=float32)


In [36]:
# # save
# with open('bilstm150_encoder_model_1124.json', 'w', encoding='utf8') as f:
#     f.write(encoder_model.to_json())
# encoder_model.save_weights('bilstm150_encoder_model_1124.h5')

# with open('bilstm150_decoder_model_1124.json', 'w', encoding='utf8') as f:
#     f.write(decoder_model.to_json())
# decoder_model.save_weights('bilstm150_decoder_model_1124.h5')

C:\Users\USER\Anaconda3\envs\gpu\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'input_3:0' shape=(?, 200) dtype=float32>, <tf.Tensor 'input_4:0' shape=(?, 200) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [63]:
# model load
from keras.models import model_from_json
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

encoder = load_model('test_bilstm150_encoder_model.json', 'test_bilstm150_encoder_model.h5')
decoder = load_model('test_bilstm150_decoder_model.json', 'test_bilstm150_decoder_model.h5')

In [65]:
reverse_input_char_index

{0: '#',
 1: '&',
 2: "'",
 3: '(',
 4: ')',
 5: '..',
 6: '1',
 7: '10',
 8: '101',
 9: '10:05',
 10: '12',
 11: '14,',
 12: '1942',
 13: "1950's",
 14: '1958',
 15: '1998.',
 16: '2',
 17: '200',
 18: '2003.',
 19: '2007-2008',
 20: '2010',
 21: '2012',
 22: '2012.',
 23: '20rh',
 24: '20th',
 25: '24',
 26: '25',
 27: '3',
 28: '3-legged',
 29: '34',
 30: '4',
 31: '4-lane',
 32: '429',
 33: '429-8044,',
 34: '4th',
 35: '4th.',
 36: '5',
 37: '50',
 38: '51',
 39: '5k',
 40: '6',
 41: '6,',
 42: '66',
 43: '70',
 44: '703',
 45: '73',
 46: '800',
 47: '8044',
 48: '80th',
 49: ':',
 50: ';',
 51: '?',
 52: '[',
 53: '[female',
 54: '[female]',
 55: "[female]'s",
 56: '[female].',
 57: '[location',
 58: '[location]',
 59: "[location]''",
 60: "[location]''.",
 61: "[location]'s",
 62: '[location],',
 63: '[location].',
 64: '[male',
 65: '[male]',
 66: "[male]'s",
 67: '[male].',
 68: '[organization',
 69: '[organization]',
 70: "[organization]''.",
 71: "[organization]'s",
 72: '[o

In [66]:
reverse_target_char_index

{0: '!',
 1: '#',
 2: '$',
 3: '&',
 4: "'",
 5: "''",
 6: '(',
 7: ')',
 8: '-',
 9: '--',
 10: '-free',
 11: '.',
 12: '..',
 13: '...',
 14: '.finally!',
 15: '0.0',
 16: '000',
 17: '1',
 18: '10',
 19: '100',
 20: "100's",
 21: '100,000',
 22: '1000',
 23: '11:30pm,',
 24: '12',
 25: '12.',
 26: '13',
 27: '1300',
 28: '1500',
 29: '1700',
 30: "1700's.",
 31: "1800's.",
 32: '1900',
 33: '1940',
 34: '1940s',
 35: '1942',
 36: '1994.',
 37: '1995.',
 38: '1998',
 39: '1st',
 40: '2',
 41: '20',
 42: '20,000',
 43: '2nd',
 44: '3',
 45: '30',
 46: '30,',
 47: '30th',
 48: '37',
 49: '3d',
 50: '3rd',
 51: '4',
 52: "40's.",
 53: '45',
 54: '4h',
 55: '4t',
 56: '4th',
 57: '4th!',
 58: '4th.',
 59: '5',
 60: '5.',
 61: '56',
 62: '5th',
 63: '66',
 64: '7th',
 65: '80',
 66: "80's",
 67: '9:20',
 68: ':',
 69: ';',
 70: '?',
 71: "?'",
 72: 'START_',
 73: '[female',
 74: '[female]',
 75: '[female]!',
 76: "[female]'s",
 77: '[female],',
 78: '[female].',
 79: '[male',
 80: '[male]

In [67]:
target_token_index['START_']

72

#### Function to generate sequences

In [68]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    #print ("target_seq: ", target_seq)
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        #print ("output_tokens: ", output_tokens)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        #print ("sampled_token_index: ", sampled_token_index)
        #print ("")
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 150): # 52
            # print ("Stop_condition = TRUE")
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

# test data

In [70]:
# load
test_input3 = ['an elderly woman hugging a young girl surrounded by other people.',
                 'couple taking a picture during a fun night in a bar.',
                 'a man is standing next to a sign for the pro football hall of fam.']

In [71]:
clean_df_all

,description_text,story_text
0,big old tree being photographed on a sunny da,"START_ and its magnificent trunk, larger than ..."
1,a old curvy tree in the sun light.,"START_ and its magnificent trunk, larger than ..."
2,a person is taking a picture of a large tree a...,"START_ and its magnificent trunk, larger than ..."
3,large tree with many outstretching branches an...,START_ we found this tree when we were walking...
4,a green sign is describing a historic tree and...,START_ it turns out it is a popular attraction...
5,a large tree with roots that look like crocodi...,"START_ the tree is very unusual, with its root..."
6,big old tree being photographed on a sunny da,"START_ the trunk was really wide, as much as 1..."
7,huge brown tree roots rose above the ground.,START_ you can see how big these roots are - p...
8,a large tree with many branches coming ou,START_ we found this tree when we were walking...
9,a plaque describes an historical tree and advi...,START_ it turns out it is a popular attraction...


In [44]:
df_input5000_0 = clean_df_all.description_text[0:5000]
df_input5000_1 = clean_df_all.description_text[5000:10000]
df_input5000_2 = clean_df_all.description_text[10000:15000]
df_input5000_3 = clean_df_all.description_text[15000:20000]
df_input5000_4 = clean_df_all.description_text[20000:25000]
df_input5000_5 = clean_df_all.description_text[25000:30000]
df_input5000_5

25000    red and blue dominate the colorful archways in...
25001       large colorful building lit up in the evening.
25002      a projection of an image made out of binary cod
25003    work of modern art being displayed and lit in ...
25004    adult man sitting on commode using laptop in w...
25005    a view of the building inside of a well-lit sc...
25006    a nighttime photograph of people outside of a ...
25007    artistic mural projections in green rendering ...
25008      a sculpture made of metal on display at a museu
25009    man sits on commode in water closet while work...
25010    the inside of the church is illuminated with d...
25011    a group of spectators is watching a a building...
25012    a red wall with a green digital display showin...
25013    an object that has been crafted by a creative ...
25014    a man sits on a toilet while using a laptop co...
25015    red and blue dominate the colorful archways in...
25016       large colorful building lit up in the evenin

In [45]:
df_story_and_desc_id

0        2626983575
1        2626983575
2        2626983575
3        2701863545
4        2626977325
5        2627795780
6        2626983575
7        2626982337
8        2701863545
9        2626977325
10       2627795780
11       2626983575
12       2626982337
13       2701863545
14       2626977325
15       2627795780
16       2626983575
17       2626982337
18       2626983575
19       2626983575
20       2626983575
21       2701863545
22       2626977325
23       2627795780
24       2626983575
25       2626982337
26       2701863545
27       2626977325
28       2627795780
29       2626983575
            ...    
29970    4794142562
29971    4794143044
29972    4793510447
29973    4793511023
29974    4793511455
29975    4794142562
29976    4794143044
29977    4793510447
29978    4793511023
29979    4793511455
29980    4794142562
29981    4794143044
29982    4793510447
29983    4793511023
29984    4793511455
29985    4794177464
29986    4793545161
29987    4794180770
29988    4793552387


In [46]:
def return_input_length(start_num, data):
    df_input_length = []
    for i in range(start_num, start_num + len(data)):
        df_input_length.append(i)
    return df_input_length

df_input5000_0_length = return_input_length(0, df_input5000_0)
df_input5000_1_length = return_input_length(5000, df_input5000_1)
df_input5000_2_length = return_input_length(10000, df_input5000_2)
df_input5000_3_length = return_input_length(15000, df_input5000_3)
df_input5000_4_length = return_input_length(20000, df_input5000_4)
df_input5000_5_length = return_input_length(25000, df_input5000_5)

df_input5000_5_length

[25000,
 25001,
 25002,
 25003,
 25004,
 25005,
 25006,
 25007,
 25008,
 25009,
 25010,
 25011,
 25012,
 25013,
 25014,
 25015,
 25016,
 25017,
 25018,
 25019,
 25020,
 25021,
 25022,
 25023,
 25024,
 25025,
 25026,
 25027,
 25028,
 25029,
 25030,
 25031,
 25032,
 25033,
 25034,
 25035,
 25036,
 25037,
 25038,
 25039,
 25040,
 25041,
 25042,
 25043,
 25044,
 25045,
 25046,
 25047,
 25048,
 25049,
 25050,
 25051,
 25052,
 25053,
 25054,
 25055,
 25056,
 25057,
 25058,
 25059,
 25060,
 25061,
 25062,
 25063,
 25064,
 25065,
 25066,
 25067,
 25068,
 25069,
 25070,
 25071,
 25072,
 25073,
 25074,
 25075,
 25076,
 25077,
 25078,
 25079,
 25080,
 25081,
 25082,
 25083,
 25084,
 25085,
 25086,
 25087,
 25088,
 25089,
 25090,
 25091,
 25092,
 25093,
 25094,
 25095,
 25096,
 25097,
 25098,
 25099,
 25100,
 25101,
 25102,
 25103,
 25104,
 25105,
 25106,
 25107,
 25108,
 25109,
 25110,
 25111,
 25112,
 25113,
 25114,
 25115,
 25116,
 25117,
 25118,
 25119,
 25120,
 25121,
 25122,
 25123,
 25124,


In [72]:
test_input3_length = return_input_length(0, test_input3)

In [73]:
df_test_input3 = pd.DataFrame(test_input3)
df_test_input3.columns = ['description_text']
df_test_input3

,description_text
0,an elderly woman hugging a young girl surround...
1,couple taking a picture during a fun night in ...
2,a man is standing next to a sign for the pro f...


In [74]:
import re
from tqdm import tqdm
# clean_df_all.description_text, clean_df_all.story_text
desc_paragraph = []
story_paragraph = []
#df_input_length_list= [df_input5000_0_length, df_input5000_1_length, df_input5000_2_length, df_input5000_3_length, df_input5000_4_length, df_input5000_5_length]
df_input_length_list = [test_input3_length]

for item in df_input_length_list: 
    for seq_index in tqdm(item):
        input_seq = encoder_input_data[seq_index: seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        #print('-')
        #print('Input sentence:', clean_df_all.description_text[seq_index: seq_index + 1])
        #print (type(clean_df_all.description_text[seq_index: seq_index + 1]))
        desc_paragraph.append(list(df_test_input3.description_text[seq_index: seq_index + 1]))
        #print('decoded sentence: ', decoded_sentence)
        re_decoded_sentence = re.sub('_END', '', decoded_sentence).strip()
        #print('Re decoded sentence:', re_decoded_sentence)
        story_paragraph.append(re_decoded_sentence)


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 21.64it/s]


In [ ]:
from nltk.translate.bleu_score import sentence_bleu

hypothesis = ['It', 'is', 'a', 'guide', 'to', 'action', 'which',
             'ensures', 'that', 'the', 'military', 'always',
             'obeys', 'the', 'commands', 'of', 'the', 'party']
reference1 = ['It', 'is', 'a', 'guide', 'to', 'action', 'that',
              'ensures', 'that', 'the', 'military', 'will', 'forever',
              'heed', 'Party', 'commands']
reference2 = ['It', 'is', 'the', 'guiding', 'principle', 'which',
              'guarantees', 'the', 'military', 'forces', 'always',
              'being', 'under', 'the', 'command', 'of', 'the',
              'Party']
reference3 = ['It', 'is', 'the', 'practical', 'guide', 'for', 'the',
              'army', 'always', 'to', 'heed', 'the', 'directions',
              'of', 'the', 'party']

sentence_bleu([reference1, reference2, reference3], hypothesis)

# temp

In [ ]:
# 11-02  decoder 1 layer
encoder_inputs = Input(shape=(None,))

display('encoder_inputs: ', encoder_inputs.shape)

en_x=  Embedding(num_encoder_tokens, EMBEDDING_DIM)(encoder_inputs)

encoder = Bidirectional(LSTM(50, return_state=True,
                             dropout = 0.5, recurrent_dropout = 0.5))

encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(en_x)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

print ("state_h: ", state_h.shape)
print ("state_c: ", state_c.shape)
print ("encoder_states: ", encoder_states)

### decoder

decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, EMBEDDING_DIM)

final_dex= dex(decoder_inputs)

print ("final_dex: ", final_dex.shape)

decoder_lstm = LSTM(100, return_sequences=True, return_state=True,
                    dropout = 0.5, recurrent_dropout = 0.5)

print ("decoder_bilstm type: ", type(decoder_lstm))

print ("decoder_bilstm: ", decoder_lstm)

#print ("encoder_states: ", encoder_states)

print ("#########")
#print ( decoder_bilstm(final_dex, initial_state=encoder_states) )
print ("#########")

decoder_outputs, _, _ = decoder_lstm(final_dex, initial_state = encoder_states)

display('decoder_outputs: ', decoder_outputs.shape)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()